<a href="https://colab.research.google.com/github/saran2811/Computer-vision/blob/main/counting_grain_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import necessary packages for image processing for rice segmentation**

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from skimage.segmentation import watershed
from skimage import filters, feature, measure, color
from scipy import ndimage


*Import train and test datasets*

In [ ]:
imagepaths = []

for dirname, _, filenames in os.walk('/content/train'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        imagepaths.append(path)

In [ ]:
imagepaths1 = []

for dirname, _, filenames in os.walk('/content/test'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        imagepaths1.append(path)

In [ ]:
fullgrain=1
brokengrain=0

In [ ]:
grain_df=pd.DataFrame({'grain_size':[],'grain_type':[]})

In [ ]:
grain_test=pd.DataFrame({'grain_size':[],'grain_type':[]})

# *Creating function for grain size dectection*

In [ ]:
def grain_size(path):
  f_gr=[]
  f_target=[]
  b_gr=[]
  b_target=[]
  image = cv2.imread(path)
  img=cv2.resize(image,(1030,700))
  grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_blur = cv2.GaussianBlur(grayscale, (3,3), 0) 
  s_x = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
  s_y = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
  s_xy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) 
  ret, thresh = cv2.threshold(img_blur, 150, 255, cv2.THRESH_BINARY)
  contours1, hierarchy1 = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  #applying watershed to solve touching grain problem and find total grains
  distance= ndimage.distance_transform_edt(thresh)

  #Applying peak_local_max function for getting coordinates of local peaks (maxima) in an image.
  local_max = feature.peak_local_max(distance, indices=False, min_distance=10, labels=thresh)
  markers = ndimage.label(local_max, structure=np.ones((3, 3)))[0]
  labels = watershed(-distance, markers, mask=thresh)
  total_gr=labels.max()
  
  #calculating full and broken grain
  for x in contours1:

  #Using contourArea method to get the area of rice (Contour)
    area = cv2.contourArea(x)
  
    if area < 100 :
     b_gr.append(area)
     b_target.append(brokengrain)
    if area >100 :
     f_gr.append(area)
     f_target.append(fullgrain)
  return f_gr,f_target,b_gr,b_target,total_gr






creating train dataframe for model creation

In [ ]:
gra_s=[]
tar=[]
test_total=[]
for i in imagepaths:
  g_size=list(grain_size(i))
  test_total.append(g_size[4])
  if len(grain_df==0):
    gra_s.append(g_size[0]+g_size[2])
    tar.append(g_size[1]+g_size[3])
  else:
    gra_s.extend(g_size[0]+g_size[2])
    
    tar.extend(g_size[1]+g_size[3])
 

<ipython-input-7-9a1e8f56618d>:20: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  local_max = feature.peak_local_max(distance, indices=False, min_distance=10, labels=thresh)


In [ ]:
grain_df['grain_size']=gra_s


In [ ]:
grain_df['grain_type']=tar

In [ ]:
grain_df.head(20)

,grain_size,grain_type
0,266.0,1
1,343.0,1
2,133.5,1
3,162.5,1
4,134.0,1
5,208.0,1
6,112.0,1
7,262.5,1
8,247.0,1
9,120.5,1


# **creating Test Dataframe for model**

In [ ]:
imagepaths1

['/content/test/image_3.jpg',
 '/content/test/image_1.jpg',
 '/content/test/image_2.jpg',
 '/content/test/image_4.jpg',
 '/content/test/image_5.jpg']

In [ ]:
brok_r=[]
gra_s1=[]
tar1=[]
test_total1=[]

for i in imagepaths1:
  g_size=list(grain_size(i))
  br_count=len(g_size[2])
  brok_r.append(br_count)
  test_total1.append(g_size[4])
  if len(grain_test==0):
    gra_s1.append(g_size[0]+g_size[2])
    tar1.append(g_size[1]+g_size[3])
    
   
  else:
    gra_s1.extend(g_size[0]+g_size[2])
    
    tar1.extend(g_size[1]+g_size[3])
 

<ipython-input-7-9a1e8f56618d>:20: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  local_max = feature.peak_local_max(distance, indices=False, min_distance=10, labels=thresh)


In [ ]:
grain_test['grain_size']=gra_s1
grain_test['grain_type']=tar1


In [ ]:
len(grain_test)

2358

In [ ]:
grain_test.head(20)

,grain_size,grain_type
0,104.5,1
1,204.0,1
2,122.0,1
3,158.5,1
4,115.5,1
5,273.0,1
6,103.0,1
7,122.5,1
8,110.5,1
9,238.5,1


In [ ]:
test_total1

[543, 505, 521, 577, 606]

In [ ]:
imagepaths1

['/content/test/image_3.jpg',
 '/content/test/image_1.jpg',
 '/content/test/image_2.jpg',
 '/content/test/image_4.jpg',
 '/content/test/image_5.jpg']

In [ ]:
output_df=pd.DataFrame({'file_name':[],'total_rice_grain':[],'total_broken_rice_grain':[]})

In [ ]:
brok_r

[288, 179, 221, 375, 179]

In [ ]:
output_df['file_name']=imagepaths1
output_df['total_rice_grain']=test_total1
output_df['total_broken_rice_grain']=brok_r

In [ ]:
output_df

,file_name,total_rice_grain,total_broken_rice_grain
0,/content/test/image_3.jpg,543,288
1,/content/test/image_1.jpg,505,179
2,/content/test/image_2.jpg,521,221
3,/content/test/image_4.jpg,577,375
4,/content/test/image_5.jpg,606,179


**Getting required output as csv file**

In [ ]:
output_df.to_csv('output_grain.csv')

In [ ]:
x_test=grain_test['grain_size'].sample(n=2000)
y_test=grain_test['grain_type'].sample(n=2000)

# **Creating Model**

Creating model and computing Accuracy using Tensorflow deep learning

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
y_train=grain_df['grain_type'].sample(n=2000)
x_train=grain_df['grain_size'].sample(n=2000)

In [ ]:
number_of_classes = 1
number_of_features = 1
model = Sequential()
model.add(Dense(number_of_classes,activation = 'sigmoid',input_dim = number_of_features))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(x_train, y_train,  validation_data=(x_test, y_test))

63/63 [==============================] - 1s 5ms/step - loss: 15.0705 - val_loss: 10.2134


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_pred=model.predict(x_test)

63/63 [==============================] - 0s 1ms/step
